In [1]:
# 異常として分類するラベル
target = 9

In [2]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

TRAIN = True

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.13, ), (0.3, ))])

trainset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)

classes = tuple(np.linspace(0, 9, 10, dtype=np.uint8))

In [3]:
raw_y = np.array([int(y) for y in trainset.targets])
raw_x = np.array([y for y in trainset.data.numpy()])

In [4]:
idx = trainset.train_labels!=target
trainset.targets = trainset.targets[idx]
trainset.data = trainset.data[idx]

/home/ken/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:45: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [5]:
idx = testset.train_labels==target
testset.targets = testset.targets[idx]
testset.data = testset.data[idx]

set dataloader

In [6]:
trainloader = torch.utils.data.DataLoader(trainset,
                                            batch_size=100,
                                            shuffle=True,
                                            num_workers=0)
testloader = torch.utils.data.DataLoader(testset, 
                                            batch_size=100,
                                            shuffle=False, 
                                            num_workers=0)

In [7]:
# 正常データをサンプル
testset_norm = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)
idx = testset_norm.train_labels!=target
testset_norm.targets = testset_norm.targets[idx]
testset_norm.data = testset_norm.data[idx]
testloader1 = torch.utils.data.DataLoader(testset_norm, 
                                            batch_size=100,
                                            shuffle=False, 
                                            num_workers=0)


## define model

In [8]:
import pretrainedmodels
model_name = "resnet18"
basemodel = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
basemodel = nn.Sequential(*list(basemodel.children())[1:-2])

In [9]:
from metrics import ArcMarginProduct
class mymodel(nn.Module):
    def __init__(self):
        super(mymodel, self).__init__()
        self.features = basemodel
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=False)
        if model_name == "resnet34" or model_name == "resnet18":
            num_ch = 512
        else:
            num_ch = 2048
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        # 全結合層で圧縮
        self.fc1 = nn.Linear(num_ch, 64)
        
    def forward(self, x):
        # extract features
        x = self.conv1(x)
        x = self.features(x)
        x = self.avgpool(x).squeeze(2).squeeze(2)
        x = F.relu(self.fc1(x))
        #x1 = self.arcface(x).squeeze(2).squeeze(2)
        
        return x

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mymodel()
model = model.to(device)

# define arcface

In [11]:
import math
NUM_OF_CLASSES = 9
class Arcface_loss(nn.Module):
    def __init__(self, feat_dim, num_classes, device="cuda", s=7.0, m=0.2):
        super(Arcface_loss, self).__init__()
        self.feat_dim = feat_dim
        self.num_classes = num_classes
        self.s = s
        self.m = m
        self.weights = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.device = device

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.mm = math.sin(math.pi-m)*m
        self.threshold = math.cos(math.pi-m)

    def forward(self, feat, label):
        eps = 1e-4
        batch_size = feat.shape[0]
        norms = torch.norm(feat, p=2, dim=-1, keepdim=True)
        feat_l2norm = torch.div(feat, norms)
        feat_l2norm = feat_l2norm * self.s

        norms_w = torch.norm(self.weights, p=2, dim=-1, keepdim=True)
        weights_l2norm = torch.div(self.weights, norms_w)

        fc7 = torch.matmul(feat_l2norm, torch.transpose(weights_l2norm, 0, 1))

        if torch.cuda.is_available():
            label = label.cuda()
            fc7 = fc7.cuda()
        else:
            label = label.cpu()
            fc7 = fc7.cpu()

        target_one_hot = torch.zeros(len(label), NUM_OF_CLASSES).to(self.device)
        target_one_hot = target_one_hot.scatter_(1, label.unsqueeze(1), 1.)        
        zy = torch.addcmul(torch.zeros(fc7.size()).to(self.device), 1., fc7, target_one_hot)
        zy = zy.sum(-1)

        cos_theta = zy/self.s
        cos_theta = cos_theta.clamp(min=-1+eps, max=1-eps) # for numerical stability

        theta = torch.acos(cos_theta)
        theta = theta+self.m

        body = torch.cos(theta)
        new_zy = body*self.s

        diff = new_zy - zy
        diff = diff.unsqueeze(1)

        body = torch.addcmul(torch.zeros(diff.size()).to(self.device), 1., diff, target_one_hot)
        output = fc7+body

        return output.to(self.device)

In [12]:
metric_fc = ArcMarginProduct(64, 9).to(device)
criterion = torch.nn.CrossEntropyLoss()

# optimzer nn
optimizer_nn = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)

# optimzer cosface or arcface
optimzer_arcface = torch.optim.SGD(metric_fc.parameters(), lr=0.01)

## train script

In [13]:
def train(epoch):
    model.train()
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    acc1 = 0.0
    acc2 = 0.0
    acc3 = 0.0
    t = tqdm(trainloader)
    
    for idx, (inputs,labels1) in enumerate(t):       
        # send to gpu
        inputs = inputs.to(device)
        labels1 = labels1.to(device)
        
        # set opt
        optimizer_nn.zero_grad()
        optimzer_arcface.zero_grad()
        
        # run model
        feat = model(inputs.float())
        outputs = metric_fc(feat, labels1)

        loss = criterion(outputs,labels1)
        loss.backward()
        
        optimizer_nn.step()
        optimzer_arcface.step()
        
        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        accuracy = (labels1.data == predicted).float().mean()
        
        if idx%100==0:
            print("loss:",loss.item())
            print("acc:", accuracy)
        

In [14]:
epochs = 6
if TRAIN:
    for epoch in range(epochs):
        train(epoch)
        torch.save(model.state_dict(), './saved_weights.pth')
else:
    model.load_state_dict(torch.load('./saved_weights.pth'))

epochs 1/6 


<ipython-input-13-81a1e1149074>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm(trainloader)


loss: 20.312061309814453
acc: tensor(0., device='cuda:0')
loss: 0.3362804651260376
acc: tensor(0.9800, device='cuda:0')
loss: 0.4617761969566345
acc: tensor(0.9600, device='cuda:0')
loss: 0.03518768399953842
acc: tensor(0.9900, device='cuda:0')
loss: 0.024246860295534134
acc: tensor(0.9900, device='cuda:0')
loss: 0.4366782307624817
acc: tensor(0.9700, device='cuda:0')

epochs 2/6 


loss: 0.15134486556053162
acc: tensor(0.9800, device='cuda:0')
loss: 1.3170379400253296
acc: tensor(0.9600, device='cuda:0')
loss: 0.329508900642395
acc: tensor(0.9800, device='cuda:0')
loss: 0.001945977215655148
acc: tensor(1., device='cuda:0')
loss: 0.3548288345336914
acc: tensor(0.9800, device='cuda:0')
loss: 0.01740581914782524
acc: tensor(0.9900, device='cuda:0')

epochs 3/6 


loss: 0.26508572697639465
acc: tensor(0.9700, device='cuda:0')
loss: 0.03718005493283272
acc: tensor(0.9900, device='cuda:0')
loss: 0.0013951110886409879
acc: tensor(1., device='cuda:0')
loss: 0.0014536285307258368
acc: tensor(1., device='cuda:0')
loss: 0.08402369171380997
acc: tensor(0.9800, device='cuda:0')
loss: 0.4465251863002777
acc: tensor(0.9700, device='cuda:0')

epochs 4/6 


loss: 0.09181439131498337
acc: tensor(0.9800, device='cuda:0')
loss: 0.06434240192174911
acc: tensor(0.9900, device='cuda:0')
loss: 0.0022960854694247246
acc: tensor(1., device='cuda:0')
loss: 0.023097658529877663
acc: tensor(0.9900, device='cuda:0')
loss: 0.3769427239894867
acc: tensor(0.9900, device='cuda:0')
loss: 0.0009610986453481019
acc: tensor(1., device='cuda:0')

epochs 5/6 


loss: 0.0012726497370749712
acc: tensor(1., device='cuda:0')
loss: 0.014887342229485512
acc: tensor(0.9900, device='cuda:0')
loss: 0.02062699757516384
acc: tensor(0.9900, device='cuda:0')
loss: 0.12098019570112228
acc: tensor(0.9800, device='cuda:0')
loss: 0.0011433553881943226
acc: tensor(1., device='cuda:0')
loss: 0.0009761953260749578
acc: tensor(1., device='cuda:0')

epochs 6/6 


loss: 0.0007634067442268133
acc: tensor(1., device='cuda:0')
loss: 0.0007749176002107561
acc: tensor(1., device='cuda:0')
loss: 0.003998646512627602
acc: tensor(1., device='cuda:0')
loss: 0.0008239984745159745
acc: tensor(1., device='cuda:0')
loss: 0.0005977201508358121
acc: tensor(1., device='cuda:0')
loss: 0.25712481141090393
acc: tensor(0.9900, device='cuda:0')



# 異常度の計算

In [15]:
from scipy import spatial
model.eval()
def cosin_metric(x1, x2):
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

In [16]:
with torch.no_grad():
    train = trainset[0][0]
    out1 = model(train.unsqueeze(0).to(device)).cpu().numpy()
    test = testset_norm[0][0]
    out2 = model(test.unsqueeze(0).to(device)).cpu().numpy()
    
print("same number:", cosin_metric(out1, out2.T))

same number: [[0.22106244]]


In [17]:
torch.cuda.empty_cache()

## inference and evaluate distance.

In [ ]:
t = tqdm(trainloader)  
for idx, (inputs,labels1) in enumerate(t):       
    # send to gpu
    inputs = inputs.to(device)
    labels1 = torch.zeros(inputs.size()[0]).to(device).long() #.unsqueeze(1)
    # run model
    with torch.no_grad():
        out = model(inputs.float())
    if idx  == 0:
        outs = out
    else:
        outs = torch.cat((outs, out))

normals = outs.cpu().numpy()

t = tqdm(testloader1)
norm_labels = []
for idx, (inputs,labels1) in enumerate(t):       
    # send to gpu
    inputs = inputs.to(device)
    norm_labels.extend(labels1.numpy())
    labels1 = torch.zeros(inputs.size()[0]).to(device).long() #.unsqueeze(1)
    # run model
    with torch.no_grad():
        out = model(inputs.float())
    if idx  == 0:
        outs = out
    else:
        outs = torch.cat((outs, out))

normals1 = outs.cpu().numpy()

t = tqdm(testloader)  
for idx, (inputs,labels1) in enumerate(t):       
    # send to gpu
    inputs = inputs.to(device)
    labels1 = torch.zeros(inputs.size()[0]).to(device).long() #.unsqueeze(1)
    # run model
    with torch.no_grad():
        out = model(inputs.float())
    if idx  == 0:
        outs = out
    else:
        outs = torch.cat((outs, out))
        
abnormals = outs.cpu().numpy()

<ipython-input-18-f7ca4b39054d>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm(trainloader)


In [ ]:
# Compute ecidian distance
with torch.no_grad():
    test = testset[1][0]
    out2 = model(test.unsqueeze(0).to(device)).cpu().numpy()

print("different number:",np.linalg.norm(normals-out2))

with torch.no_grad():
    test = testset_norm[1][0]
    out2 = model(test.unsqueeze(0).to(device)).cpu().numpy()

print("same number:", np.linalg.norm(normals-out2))

In [ ]:
# compute cosine distance with my function
with torch.no_grad():
    test = testset[2][0]
    out2 = model(test.unsqueeze(0).to(device)).cpu().numpy()

print("different number:", 1-np.mean(cosin_metric(normals, out2.T)))

with torch.no_grad():
    test = testset_norm[2][0]
    out2 = model(test.unsqueeze(0).to(device)).cpu().numpy()

print("same number:", 1-np.mean(cosin_metric(normals, normals.T), axis=0))

In [ ]:
# compute cosine distance with scipy
with torch.no_grad():
    test = testset[2][0]
    out2 = model(test.unsqueeze(0).to(device)).cpu().numpy()
print("different number:", 1 - spatial.distance.cosine(normals[0,:], out2.T))

In [ ]:
# print scatter plot of same 1s
# Euclidian
norm = []
for n in normals1:
    norm.append(np.linalg.norm(normals-n))
import matplotlib.pyplot as plt
same = norm
plt.plot(same)
plt.show()

In [ ]:
# print scatter plot of same 1s
# Euclidian
norm = []
for n in abnormals:
    norm.append(np.linalg.norm(normals-n))
import matplotlib.pyplot as plt
different = norm
plt.plot(different)
plt.show()

# 推論結果を3D可視化

In [ ]:
# https://github.com/egcode/pytorch-losses/blob/master/mnist-visualize-arcface6_fc7-loss.ipynb
f3d = []
lbls = []
for i in range(len(testset_norm)):
    image_tensor, label_tensor = testloader1.dataset[i]
    label_tensor = torch.tensor(testloader1.dataset[i][1])
    image_tensor = image_tensor.reshape(1,1,28,28)

    image_tensor, label_tensor = image_tensor.to(device), label_tensor.to(device)

    features3d  = model(image_tensor)
    logits = metric_fc(features3d, torch.unsqueeze(label_tensor, dim=-1))
    _, prediction = torch.max(logits.data, 1)

    f3d.append(features3d[0].cpu().detach().numpy())
    
    prediction = prediction.cpu().detach().numpy()[0]
    lbls.append(prediction)

feat3d = np.array(f3d)
print("3d features shape" + str(feat3d.shape))

lbls = np.array(lbls)
print("labels shape" + str(lbls.shape))

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(16,9))
ax = plt.axes(projection='3d')

for i in range(10):
    # Data for three-dimensional scattered points
    xdata = feat3d[lbls==i,2].flatten()
    ydata = feat3d[lbls==i,0].flatten()
    zdata = feat3d[lbls==i,1].flatten()
    ax.scatter3D(xdata, ydata, zdata);
ax.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

# Visualize by TSNE

## 普通の分類結果

In [ ]:
from sklearn.manifold import TSNE
# 特徴量抽出
feats = normals1
# ラベル情報追加

# TSNEをfit
tSNE_metrics = TSNE(n_components=2, random_state=0).fit_transform(feats)
plt.scatter(tSNE_metrics[:, 0], tSNE_metrics[:, 1], c=norm_labels)
plt.colorbar()
plt.savefig('tsne_classification.png')
plt.show()

## 異常ラベルとの比較

In [ ]:
predict_y = np.concatenate([normals1, abnormals])
diflabel = np.ones_like(different)*9
test_y = np.concatenate([norm_labels, diflabel])

tSNE_metrics = TSNE(n_components=2, random_state=0).fit_transform(predict_y)
plt.scatter(tSNE_metrics[:, 0], tSNE_metrics[:, 1], c=test_y)
plt.colorbar()
plt.savefig('tsne_abnormals.png')
plt.show()

## plot roc curve

In [ ]:
predict_y = np.concatenate([same, different])
samelabel = np.ones_like(same)
diflabel = np.zeros_like(different)
test_y = np.concatenate([samelabel, diflabel])

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

# FPR, TPR(, しきい値) を算出
fpr, tpr, thresholds = metrics.roc_curve(test_y, predict_y)

In [ ]:
# ついでにAUCも
auc = metrics.auc(fpr, tpr)

# ROC曲線をプロット
plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)